In [24]:
%reload_kedro

2020-06-18 13:40:33,692 - root - INFO - ** Kedro project Melanoma
2020-06-18 13:40:33,693 - root - INFO - Defined global variable `context` and `catalog`
2020-06-18 13:40:33,719 - root - INFO - Registered line magic `run_viz`


In [2]:
import torch
from kedro.context import load_context

In [3]:
project_context = load_context('/home/michal/Documents/kaggle_projects/melanoma/')


In [18]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
#import torch.utils.data as data
#from torch.utils.data import Dataset
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
import os
%matplotlib inline

In [26]:

train_dir = project_context.catalog.load('test_raw')
test_dir = project_context.catalog.load('test_raw')

2020-06-18 13:43:18,309 - kedro.io.data_catalog - INFO - Loading data from `test_raw` (ImageDataSet)...
2020-06-18 13:43:18,326 - kedro.io.data_catalog - INFO - Loading data from `test_raw` (ImageDataSet)...


In [27]:
batch_size = 32
train_loader = DataLoader(train_dir, batch_size=batch_size, shuffle=True,num_workers=2)
data_iter = iter(train_loader)
images= data_iter.next()
print(images)

[tensor([[[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]],

         [[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]],

         [[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, 

In [14]:
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image
# obtain one batch of training images
data_iter = iter(train_loader)
images, lbls = data_iter.next()
images = images.numpy() # convert images to numpy for display
# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(10, 4))
# display 20 images



for idx in np.arange(10):
    ax = fig.add_subplot(2, 10/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    label = lbls[idx]
    #ax.set_title(classes[label])
    ax.set_title(classes[lbls[idx]])


train_loader.batch_size()

TypeError: 'int' object is not callable

In [ ]:
#Defining transformations for training and test data
#transforms.compose() will apply transformation to images
transformation = transforms.Compose([transforms.RandomHorizontalFlip(),
                                     transforms.RandomRotation(20),
                                     transforms.Resize(size=(224,224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                     ])
#Load the dataset with Image Folder
trainset = datasets.ImageFolder(train_dir, transform = transformation)
testset = datasets.ImageFolder(test_dir, transform = transformation)

#define data loaders
batch_size = 32
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True,num_workers=2)
test_loader = DataLoader(testset, batch_size=batch_size,num_workers=1)

In [6]:
from kedro.config import ConfigLoader, MissingConfigException
conf_paths = ["conf/base", "conf/local"]
conf_loader = ConfigLoader(conf_paths)

try:
    parameters = conf_loader.get("parameters*", "parameters*/**")
except MissingConfigException:
    parameters = {}


In [7]:

print(parameters['transformations']['resize']['height'])

224


In [14]:
x = (parameters['transformations']['resize']['height'], parameters['transformations']['resize']['width'])


In [16]:
type(x)

tuple